# The goals are:

- Document the steps involved in assessing liquidations in Defi lending
- Create a function that assesses the 'health' of a loan 
- Create a workflow that uses a market trigger to 'post margin' to prevent users from liquidations

## 1. What metric(s) do defi lending protocols use to measure the 'health' of a loan?

### 1.1 Crypto-Liquidation Value (CLV)

#### 1.1a CLV: Notes

one way to measure the sensitivity to liquidation for a loan backed by a cryptocurrency is to calculate the "crypto-liquidation value" (CLV). The CLV is the percentage decrease in the price of the cryptocurrency that would result in the loan being fully liquidated (i.e., the value of the collateral is equal to or less than the outstanding loan balance).

The formula for CLV is:

CLV = (LTV - 1) / (1 - MC)

Where:

- LTV is the loan-to-value ratio, expressed as a decimal (e.g., 0.5 for a 50% LTV. The loan-to-value ratio is the ratio of the loan amount to the value of the collateral.
- MC is the maintenance collateral, expressed as a decimal (e.g., 0.15 for a 15% maintenance collateral). The maintenance collateral is the minimum collateral value required to maintain the loan. If the value of the collateral falls below the maintenance collateral, the lender may initiate a liquidation of the collateral to recover the loan amount. For example, if the maintenance collateral is set at 15%, the value of the collateral must be at least 15% higher than the outstanding loan balance to avoid liquidation.
 
The CLV formula calculates the percentage decrease in the price of the cryptocurrency that would result in the loan being fully liquidated. For example, if the CLV is 30%, a 30% decrease in the price of the cryptocurrency would result in the value of the collateral being equal to or less than the outstanding loan balance, triggering liquidation.

It is my understanding that LTV must be =< 0.67 (150% colateral)

We will need to find out each protocols MC is in order to calculate CLV

#### 1.1b CLV: Code

In [7]:
def clv(ltv, mc):
    """
    Calculates the crypto-liquidation value (CLV) for a loan backed by a cryptocurrency.

    Parameters:
    ltv (float): The loan-to-value ratio, expressed as a decimal.
    mc (float): The maintenance collateral, expressed as a decimal.

    Returns:
    float: The crypto-liquidation value, expressed as a percentage.
    """
    clv = (ltv - 1) / (1 - mc) * 100  # Calculate the CLV as a percentage
    return clv

In [8]:
# Example

ltv = 0.8  # 50% LTV
mc = 0.05  # 15% maintenance collateral
clv = clv(ltv, mc)
print(f"Crypto-liquidation value: {clv:.2f}%")

Crypto-liquidation value: -21.05%


#### 1.1c CLV: Data

- No data to be queried for CLV

- We should be able to set MC ourselves for each protocol once we find out what it is

- LTV should be a variable to be set by the customer 

### 1.2 Volatility

#### 1.2a Volatility: Notes

The CLV returns the % drawndown that would cause liquidation. The workflow needs to act before this value. 

The formula for calculating the variance threshold (VT) is as follows:

Calculate the mean:
Mean = SMA (Closing Price, n)

Where:

Closing Price is the closing price of the financial instrument over a specified time period (e.g., the last 20 trading days).
n is the number of time periods used to calculate the SMA (e.g., 20 days).

Calculate the variance threshold:
Lower Band = Mean - (k * Standard Deviation)

Where:

Standard Deviation is a measure of the dispersion of the closing prices around the SMA.
k is the number of standard deviations to add or subtract from the middle band to obtain the upper and lower bands. The most common value for k is 2.

#### 1.2b Volatility: Code

In [9]:
import numpy as np

def var_threshold(data, n, k):
    """
    Calculates a value k standard deviations lower than a simple moving average (SMA) of the price of a token.

    Parameters:
    data (numpy.ndarray): An array of token prices.
    n (int): The number of time periods used to calculate the SMA.
    k (int): The number of standard deviations to subtract from the SMA.

    Returns:
    float: The var_threshold value.
    """
    sma = np.mean(data[-n:])  # Calculate the SMA of the last n prices
    std = np.std(data[-n:])   # Calculate the standard deviation of the last n prices
    var_threshold = sma - (k * std)  # Calculate the lower band as k standard deviations below the SMA
    return var_threshold

In [10]:
# Example

data = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48]  # Example token prices
n = 20  # Calculate the 20-day SMA 
k = 2   # Calculate the var_threshold that is 2 standard deviations below the SMA
var_threshold = var_threshold(data, n, k)
print(f"The {n}-day {k}-sigma var_threshold is: {var_threshold:.2f}")

The 20-day 2-sigma var_threshold is: 5.93


#### 1.2c Volatility: Data

- Pricing data needed (OHLC, really just C)
    - Historical pricing data (SMA needs to be calculated using n periods back where the length of a period is set by the user)
    - Live price feeds needed to update van_threshold and trigger workflow 

## Random / Uncategorised 

Three things to conider are:
- The Liquidation Prevention Threshold (LPT) to trigger the workflow (e.g. in the example above Crypto-liquidation value is -58.82%, therefore: -58.82 < LPT < 0, Generalized: CLV < LPT < 0). This will be a variable set by the user.
- The LTV after margin has been posted (e.g. once the LPT has been met and the workflow is triggered the amount of new colateral must be calulated to reach the desired LTV)
- Volatility 

#### Start
Set params:
- Loan-to-value ratio (LTV)				User
- Maintenance collateral (MC)				Protocol
- Liquidation Prevention Threshold (LPT)		User
- Loan account 						User
- Margin account					User
- Repeat bool						User
- Time interval to query data				User/FMP

#### Initialize 
Verify settings:
- Crypto-Liquidation Value (CLV) 				
- Verify loan account					
- Verify margin account
- What happens if this account is active and falls below what is required to prevent liquidation?

#### Idle
Query:
- Pricing data (OHLC)
- Margin account

#### Trigger 
Complete workflow:
- Send x amount from margin account to loan account

#### End | Start

